In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/emotion project/training',
    image_size=(224, 224),
    batch_size=64,
    seed=42,
    color_mode='rgb'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/emotion project/val',
    image_size=(224, 224),
    batch_size=64,
    seed=42,
    color_mode='rgb'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/emotion project/test',
    image_size=(224, 224),
    batch_size=64,
    seed=42,
    color_mode='rgb'
)


def preprocess_func(image, label):
    image = tf.cast(image / 255.0, tf.float32)
    return image, label


train_ds = train_ds.map(preprocess_func)
val_ds = val_ds.map(preprocess_func)
test_ds = test_ds.map(preprocess_func)





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1527 files belonging to 4 classes.
Found 374 files belonging to 4 classes.
Found 390 files belonging to 4 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(4, activation='softmax'))
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

def train_model(model, train_data, val_data, epochs):
    # 모델 학습
    history = model.fit(train_data, validation_data=val_data, epochs=epochs, verbose=1)
    return history

history = train_model(model, train_ds, val_ds, epochs=5)


test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")